In [1]:
import sys
sys.path.insert(0, '..')
import pickle

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import gym

%matplotlib inline

In [2]:
import tf_util
import load_policy

In [3]:
policy_fn = load_policy.load_policy('../experts/Hopper-v1.pkl')

obs (1, 11) (1, 11)


In [4]:
sess = tf.InteractiveSession()

### Generate training data

In [5]:
num_rollouts = 10
max_timesteps = 2000

# Use the first 10000 (obs, action) pairs as training data
env = gym.make('Hopper-v1')
X_all, y_all = [], []

for i in range(num_rollouts):
    print(i, end=',')
    obs = env.reset()
    action = policy_fn(obs[None, :])
    X_all.append(obs)
    y_all.append(action)   
    for j in range(max_timesteps):
        obs, r, done, _ = env.step(action)
        action = policy_fn(obs[None, :])
        X_all.append(obs)
        y_all.append(action)
        
X_all = np.array(X_all)
# concat instead of array is necessary to keep array in the right shape
y_all = np.concatenate(y_all)

0,1,2,3,4,5,6,7,8,9,

In [6]:
# split index
idx = int(X_all.shape[0] * 4/5)

X_tv = X_all[:idx, :]
X_test = X_all[idx:, :]
y_tv = y_all[:idx, :]
y_test = y_all[idx:, :]

In [7]:
print(X_tv.shape, X_test.shape, y_tv.shape, y_test.shape)

(16008, 11) (4002, 11) (16008, 3) (4002, 3)


In [8]:
with open('train_test.pkl', 'wb') as opf:
    pickle.dump([X_tv, y_tv, X_test, y_test], opf)